In [2]:
from llama_cpp import Llama
import os
import urllib.request

In [7]:
def download_file(file_link, filename):
    # Checks if the file already exists before downloading
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(file_link, filename)
        print("File downloaded successfully.")
    else:
        print("File already exists.")

# Dowloading GGML model from HuggingFace
ggml_model_path = "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/blob/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf"
filename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"

download_file(ggml_model_path, filename)

File already exists.


In [3]:
llm = Llama(model_path="mistral-7b-instruct-v0.2.Q5_K_M.gguf", n_ctx=256, n_batch=126)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 l

llama_model_loader: - kv  13:                      tokenizer.ggml.tokens arr[str,32000]   = ["<unk>", "<s>", "</s>", "<0x00>", "<...
llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  15:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  16:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  17:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  18:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  19:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  20:               tokenizer.ggml.add_bos_token bool             = true
llama_model_loader: - kv  21:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  22: 

In [4]:
def generate_text(
        prompt="Who is the CEO of Apple?",
        max_tokens=512,
        temperature=0.4,
        top_p=0.5,
        echo=False,
        stop=["#"],
):
    output = llm(prompt,
                 max_tokens=max_tokens,
                 temperature=temperature,
                 top_p=top_p,
                 echo=echo,
                 stop=stop,
                 )
    output_text = output["choices"][0]["text"].strip()
    return output_text

In [14]:
def generate_prompt_from_template(input):
    chat_prompt_template =f"""user
You are receiving a scam email and want to reply to waste the scammer's time. Craft a response to the email below:
{input}
system
Feel free to engage, ask irrelevant questions, act confused, or provide false information to waste their time. Provide only the body of the message. Keep it short."""    
    return chat_prompt_template

In [15]:
prompt = generate_prompt_from_template("Our system indicates that there has been suspicious activity detected on your account and we require your immediate attention to verify your account information to prevent any unauthorized access. Please click on the following link to proceed with the verification process: Failure to verify your account within the next 24 hours may result in temporary suspension or permanent closure of your account. ")

generate_text(prompt=prompt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   10305.68 ms
llama_print_timings:      sample time =      18.86 ms /   111 runs   (    0.17 ms per token,  5885.78 tokens per second)
llama_print_timings: prompt eval time =    2553.46 ms /    30 tokens (   85.12 ms per token,    11.75 tokens per second)
llama_print_timings:        eval time =   17969.29 ms /   110 runs   (  163.36 ms per token,     6.12 tokens per second)
llama_print_timings:       total time =   20857.36 ms /   140 tokens


'I\'m sorry, I don\'t have my mouse with me right now. Could you please send me the verification link via text message instead? And what exactly do you mean by "account information"? Is it my username, password, or both? I also noticed you mentioned "temporary suspension" or "permanent closure" of my account. What happens if my account is closed? Will I be able to open a new one? Please clarify these points before I proceed with the verification process. Thank you for your understanding.'